In [2]:
from qutip import *
import cmath
import math
import matplotlib.pyplot as plt
import numpy as np

In [17]:
nqubits = 3

init = []
for _ in range(nqubit):
    init.append(basis(2,0))

inital_state = tensor(init)

In [18]:
inital_state

Quantum object: dims=[[2, 2, 2], [1, 1, 1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]

In [19]:
# 2^n dim ideentity operator, n: number of qubits
identity_list = []
for _ in range(nqubit):
    identity_list.append(qeye(2))
Qeye = tensor(identity_list)

In [20]:
Qeye

Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', dtype=Dia, isherm=True
Qobj data =
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]

In [21]:
# X, Y, Z operator

def Xop(m: int, nqubits: int) -> object:
    """
    Constructs the Pauli X operator acting on the m-th qubit 
    in a quantum system of n total qubits.

    Args:
        m (int): The index of the target qubit (0-based).
        nqubits (int): The total number of qubits in the system.

    Returns:
        object: The tensor product operator with the Pauli X operator 
                applied to the m-th qubit and identity operators on 
                all other qubits.
    """

    operators = []

    for n in range(nqubits):
        if n == m:
            operators.append(sigmax())
        else:
            operators.append(qeye(2))

    return tensor(operators)

def Yop(m: int, nqubits: int) -> object:
    """
    Constructs the Pauli Z operator acting on the m-th qubit 
    in a quantum system of n total qubits.

    Args:
        m (int): The index of the target qubit (0-based).
        nqubits (int): The total number of qubits in the system.

    Returns:
        object: The tensor product operator with the Pauli X operator 
                applied to the m-th qubit and identity operators on 
                all other qubits.
    """

    operators = []

    for n in range(nqubits):
        if n == m:
            operators.append(sigmaz())
        else:
            operators.append(qeye(2))

    return tensor(operators)

def Zop(m: int, nqubits: int) -> object:
    """
    Constructs the Pauli Z operator acting on the m-th qubit 
    in a quantum system of n total qubits.

    Args:
        m (int): The index of the target qubit (0-based).
        nqubits (int): The total number of qubits in the system.

    Returns:
        object: The tensor product operator with the Pauli X operator 
                applied to the m-th qubit and identity operators on 
                all other qubits.
    """

    operators = []

    for n in range(nqubits):
        if n == m:
            operators.append(sigmaz())
        else:
            operators.append(qeye(2))

    return tensor(operators)
    

In [25]:
# Hamiltonian

def construct_hami(nqubits:int, v: float, hx: float, hz: float) -> object:

    hamiltonian = []
    for n in range(nqubits-1):
        hamiltonian.append(v/4 * Zop(n, nqubit)*Zop(n+1, nqubits))
    for n in range(nqubits):
        hamiltonian.append(hx * Xop(n, nqubits)*Xop(n, nqubits))
        hamiltonian.append(hz * Zop(n, nqubits)*Zop(n, nqubits))
    
    # Initialize the Hamiltonian as a zero operator
    Hamil = 0 * Zop(0, nqubits)

    for term in hamiltonian:
        Hamil += term
    
    return Hamil